In [1]:
%load_ext autoreload
%autoreload 2
import os
import torch
import numpy as np
from glob import glob
from tqdm.auto import tqdm
from sklearn.linear_model import LogisticRegression
import pandas as pd
from utils.store_data import ReadData
import numpy as np
import torch
from utils.arguments import Arguments
from tqdm import tqdm
from sklearn.decomposition import PCA
from linear_probe import get_linear_acc
import random
import os
from sklearn.metrics import roc_auc_score,accuracy_score,roc_curve,f1_score
from ml_utils.metrics import aur_pr
from answer_judge import load_judge

In [2]:
data_train=ReadData('/root/autodl-tmp/hallscope/AOKVQA/train').read_all()
random.shuffle(data_train)
#data_val=ReadData('/root/autodl-fs/hallscope/ScienceQA/val').read_all()
#data_test=ReadData('/root/autodl-fs/hallscope/ScienceQA/test').read_all()

In [3]:
df_train=pd.DataFrame(data_train)
#df_val=pd.DataFrame(data_val)
#df_test=pd.DataFrame(data_test)

In [4]:
df_train.head()

,img_path,question,answer,most_likely,responses
0,[/root/autodl-tmp/coco2017/train2017/000000079...,[Question:What is the best climate for these f...,"[tensor(3), tensor(1), tensor(3), tensor(0), t...",{'embedding': [[[-0.02514648 0.01312256 0.00...,None
1,[/root/autodl-tmp/coco2017/train2017/000000456...,[Question:What type of area is this event taki...,"[tensor(1), tensor(3), tensor(3), tensor(0), t...",{'embedding': [[[ 7.2813034e-04 6.3419342e-04...,None
2,[/root/autodl-tmp/coco2017/train2017/000000398...,[Question:How is the zebra decorated?\nAnswer ...,"[tensor(0), tensor(0), tensor(3), tensor(3), t...",{'embedding': [[[ 0.00479126 0.00378418 -0.00...,None
3,[/root/autodl-tmp/coco2017/train2017/000000042...,[Question:How many cows are evidently in the p...,"[tensor(2), tensor(1), tensor(1), tensor(0), t...",{'embedding': [[[ 7.2813034e-04 6.3419342e-04...,None
4,[/root/autodl-tmp/coco2017/train2017/000000488...,[Question:What are the rugs for?\nAnswer the Q...,"[tensor(2), tensor(1), tensor(0), tensor(2), t...",{'embedding': [[[ 7.2813034e-04 6.3419342e-04...,None


In [5]:
i=0
df_train.iloc[i]['most_likely']['response']

['D.tropical', 'A', 'D', 'A.porch swing', 'D', 'B.bus depot', 'A.camp', 'D']

In [6]:
df_train.iloc[i]['img_path']

['/root/autodl-tmp/coco2017/train2017/000000079992.jpg',
 '/root/autodl-tmp/coco2017/train2017/000000548485.jpg',
 '/root/autodl-tmp/coco2017/train2017/000000246686.jpg',
 '/root/autodl-tmp/coco2017/train2017/000000175142.jpg',
 '/root/autodl-tmp/coco2017/train2017/000000147442.jpg',
 '/root/autodl-tmp/coco2017/train2017/000000334675.jpg',
 '/root/autodl-tmp/coco2017/train2017/000000398772.jpg',
 '/root/autodl-tmp/coco2017/train2017/000000273379.jpg']

In [7]:
df_train.iloc[i]['question']

['Question:What is the best climate for these fruits to grow in?\nAnswer the Question with following choices.\nChoices: A.dry B.arid C.polar D.tropical\n',
 'Question:Why are the cleaning bottles on the tub wall?\nAnswer the Question with following choices.\nChoices: A.visual appeal B.convivence C.accidental D.safety\n',
 'Question:What is the man wearing the hat pulling?\nAnswer the Question with following choices.\nChoices: A.surfboard B.snow board C.door D.body board\n',
 'Question:What is the swinging bench called?\nAnswer the Question with following choices.\nChoices: A.porch swing B.hanging swing C.dangerous D.outdoor loveseat\n',
 'Question:What is the photographer definitely higher than?\nAnswer the Question with following choices.\nChoices: A.people B.clouds C.tower D.air plane\n',
 'Question:What location is this?\nAnswer the Question with following choices.\nChoices: A.zoo B.bus depot C.subway D.carnival\n',
 'Question:What is this place?\nAnswer the Question with following 

In [8]:
df_train.iloc[0]['answer']

tensor([3, 1, 3, 0, 0, 1, 1, 1])

In [9]:
embeddings=df_train['most_likely'].apply(lambda x: x['embedding']).to_list()
embeddings=np.concatenate(embeddings,axis=0)
embeddings.shape

(17056, 33, 4096)

In [10]:
gt_label=df_train['answer'].to_list()
gt_label=np.concatenate(gt_label,axis=0)
gt_label.shape

(17056,)

In [11]:
gt_label[0]

np.int64(3)

In [12]:
judge=load_judge('OKVQA')

In [13]:
responses=df_train['most_likely'].apply(lambda x: x['response']).to_list()

rsps=[]
for x in responses:
    rsps+=x
    
is_correct=[]
for i,x in enumerate(rsps):
    is_correct.append(judge.check(x,gt_label[i]))

is_correct=np.array(is_correct)
print(f'There are {len(is_correct)} responses. {sum(is_correct==1)} answer correctly and {sum(is_correct==0)} answer wrongly')

There are 17056 responses. 11081 answer correctly and 5975 answer wrongly


In [14]:
length = len(embeddings)
train_rate = 0.8
num_val=500

embedding_train = embeddings[:int(length*train_rate)-num_val]
embedding_val = embeddings[int(length*train_rate)-num_val:int(length*train_rate)]
embedding_test = embeddings[int(length*train_rate):]


y_train = is_correct[:int(length*train_rate)-num_val]
y_val = is_correct[int(length*train_rate)-num_val:int(length*train_rate)]
y_test = is_correct[int(length*train_rate):]


'''get testset, wildset and valset. The valset is used for determining the hype-parameters'''
print(f'trainset length:{len(y_train)}')
print(f'validset length:{len(y_val)}')
print(f'testset length:{len(y_test)}')
print(
    f'trainset postive: {sum(y_train==1)} negative: {sum(y_train==0)}')
print(
    f'valset postive: {sum(y_val==1)} negative: {sum(y_val==0)}')
print(
    f'testset postive: {sum(y_test==1)} negative: {sum(y_test==0)}')

trainset length:13144
validset length:500
testset length:3412
trainset postive: 8509 negative: 4635
valset postive: 303 negative: 197
testset postive: 2269 negative: 1143


In [15]:
%load_ext autoreload 
%autoreload 2
from ml_utils.grid_search import GridSearch
from ml_utils.PCA_discriminator import PCADiscriminator
from ml_utils.metrics import auroc,get_best_split_from_scores

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Grid Search for Best 'n_components' & 'layer'

In [16]:
def evaluator(n_components,i_layer,X,y):
    discriminator=PCADiscriminator(n_components,X[:,i_layer,:])
    scores=discriminator.get_score()
    return auroc(scores,y)

In [17]:
# graid search for best hyper-parameters on validation set
grid={
    'n_components':range(1,12),
    'layer':range(embedding_val.shape[1])
}
grid_search=GridSearch(evaluator,grid,embedding_val,y_val)
best_paras=grid_search.search()

Grid Searching for best n_components,layer


  0%|          | 0/11 [00:00<?, ?it/s]

found best n_components:1,layer:0 Score: 0.5864619456869544


  9%|▉         | 1/11 [00:00<00:07,  1.31it/s]

found best n_components:2,layer:8 Score: 0.592601899783887


100%|██████████| 11/11 [00:13<00:00,  1.22s/it]


In [18]:
best_paras

{'n_components': 2, 'layer': 8, 'best_score': np.float64(0.592601899783887)}

In [19]:
best_layer=best_paras['layer']

View Results on Training set and Test set

In [20]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_val[:,best_layer,:])
best_split=discriminator.get_best_split(y_val)

In [21]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_train[:,best_layer,:])
scores=discriminator.get_score()
preds=(scores>best_split)
print(f'train set acc: {accuracy_score(y_train,preds)} AUROC:{roc_auc_score(y_train,scores)} AUC-PR:{aur_pr(scores,y_train)}')

train set acc: 0.6409007912355448 AUROC:0.5121280938274253 AUC-PR:0.6570480748656433


In [22]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_test[:,best_layer,:])
scores=discriminator.get_score()
preds=(scores>best_split)
print(f'test set acc: {accuracy_score(y_test,preds)} AUROC:{roc_auc_score(y_test,scores)} AUC-PR:{aur_pr(scores,y_test)}')

test set acc: 0.6635404454865181 AUROC:0.47463067777611984 AUC-PR:0.653407256968942


Train Linear Prob to Get Better Results

In [23]:
X_train=embedding_train[:, best_layer,:]
y_train=y_train

X_test=embedding_test[:, best_layer,:]
y_test=y_test

In [24]:
best_acc, final_acc, (
            clf, best_state, best_preds, preds, labels_val), losses_train = get_linear_acc(
            X_train,
            y_train,
           X_train,
            y_train,
            2, epochs=50,
            print_ret=True,
            batch_size=512,
            cosine=True,
            nonlinear=True,
            learning_rate=0.05,
            weight_decay=0.0003)

In [25]:
clf.eval()
output = clf(torch.from_numpy(X_test).to(torch.float32).cuda())
scores = torch.sigmoid(output).cpu().data.numpy()
preds=scores>0.5
print(f'test set acc: {accuracy_score(y_test,preds)} AUROC:{roc_auc_score(y_test,scores)} AUC-PR:{aur_pr(scores,y_test)} f1:{f1_score(y_test,preds)}')

test set acc: 0.6796600234466589 AUROC:0.6719638229443443 AUC-PR:0.7864010401345312 f1:0.784120086904997
